# Notebook dùng để crawl thông tin các user 

### Import các thư viện cần dùng

In [1]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv

### Setup trước khi crawl

In [2]:
sleep_time = 1 # Đơn vị: giây

In [3]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

### Định nghĩa hàm crawl dữ liệu user

In [4]:
def get_user_profile(user_id_file, client_id):
    '''
    Thu thập dữ liệu các user tương ứng trong user_id_file
    
    Parameters
    -------
    user_id_file : đường dẫn file .txt chứa danh sách user ID cần 
                   thu thập dữ liệu
    client_id : ID truy cập API soundcloud
    
    Returns
    -------
    playlist : DataFrame của các user thu được
    '''
    
    list_user_id = np.loadtxt(user_id_file, dtype=int) # User id list
    
    # Khởi tạo user dataframe
    user = pd.DataFrame(columns = ['id', 'username', 'city', 'country_code', 'first_name', 'last_name', 'created_at', 
                                   'followers_count', 'followings_count', 'playlist_count','track_count','verified', 
                                   'likes_count', 'comments_count', 'playlist_likes_count',
                                   'last_modified', 'avatar_url','permalink_url'])
    
    # Các Attributes trong user dataframe
    attributes = ['id', 'username', 'city', 'country_code', 'first_name', 'last_name', 'created_at', 
                    'followers_count', 'followings_count', 'playlist_count','track_count','verified', 
                    'likes_count', 'comments_count', 'playlist_likes_count',
                    'last_modified', 'avatar_url','permalink_url']

    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[403, 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))
    
    # Duyệt từng user id
    for user_id in list_user_id:    
        # Lấy request với user_id tương ứng
        url = f'https://api-v2.soundcloud.com/users/{user_id}?client_id={client_id}'
        # Ghi log để theo dõi
        print('Get:', url)
        print('Status: Doing')
        
        try:
            re = s.get(url)
        except Exception as e:
            # Bỏ qua việc thu thập nếu gặp phải lỗi hoặc chờ quá lâu
            if e != KeyboardInterrupt:
                print("Error or timeout! Status code:",re.status_code)
                print("Ignore this user")
                continue
        
        # Lặp liên tục đến khi request được
        canGet = False
        while not canGet:
            # Sleep rồi request lại
            if not re.ok:
                print('Status: error', re.status_code)
                time.sleep(sleep_time) 
                re = s.get(url)       
                continue
            
            if re.status_code == 429:
                print('Status: Exceeds an API rate limit')
                print("================Stop crawling=================")
                return user
                
            # Convert Json string thành python dictionary
            json_pydata = json.loads(re.text)
            
            # Request đã xong
            canGet = True
          
        # Ghi log
        print('Status: Done')
          
        # Temporary dictionary để lưu các playlist record hiện tại
        temp = {}
            
        # Lấy thuộc tính tương ứng vào temp
        for key in attributes:
            temp[key] = json_pydata[key]
            
        # Mở rộng tập dữ liệu vừa thu được
        user = user.append(temp, ignore_index=True)
        
        # Ghi log
        print('Total user records:', len(user))
        
    return user

### Tiến hành thu thập dữ liệu

In [5]:
user = get_user_profile('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

Get: https://api-v2.soundcloud.com/users/630253005?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 1
Get: https://api-v2.soundcloud.com/users/217441590?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 2
Get: https://api-v2.soundcloud.com/users/324531068?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 3
Get: https://api-v2.soundcloud.com/users/525378972?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 4
Get: https://api-v2.soundcloud.com/users/9855085?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 5
Get: https://api-v2.soundcloud.com/users/732907996?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 6
Get: https://api-v2.soundcloud.com/users/311417284?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 7
G

Status: Done
Total user records: 72
Get: https://api-v2.soundcloud.com/users/17603266?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 73
Get: https://api-v2.soundcloud.com/users/427999?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 74
Get: https://api-v2.soundcloud.com/users/10802529?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 75
Get: https://api-v2.soundcloud.com/users/2495919?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 76
Get: https://api-v2.soundcloud.com/users/83277618?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 77
Get: https://api-v2.soundcloud.com/users/23951437?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 78
Get: https://api-v2.soundcloud.com/users/99008?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Statu

Total user records: 142
Get: https://api-v2.soundcloud.com/users/225675436?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 143
Get: https://api-v2.soundcloud.com/users/599232756?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 144
Get: https://api-v2.soundcloud.com/users/700166945?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 145
Get: https://api-v2.soundcloud.com/users/1448991?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 146
Get: https://api-v2.soundcloud.com/users/813082876?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 147
Get: https://api-v2.soundcloud.com/users/514745913?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 148
Get: https://api-v2.soundcloud.com/users/100605233?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing


Total user records: 205
Get: https://api-v2.soundcloud.com/users/909658033?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 206
Get: https://api-v2.soundcloud.com/users/14168495?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 207
Get: https://api-v2.soundcloud.com/users/10958157?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 208
Get: https://api-v2.soundcloud.com/users/65914246?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 209
Get: https://api-v2.soundcloud.com/users/450856584?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 210
Get: https://api-v2.soundcloud.com/users/270614338?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 211
Get: https://api-v2.soundcloud.com/users/459880035?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
S

Status: Done
Total user records: 266
Get: https://api-v2.soundcloud.com/users/523496169?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 267
Get: https://api-v2.soundcloud.com/users/140016384?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 268
Get: https://api-v2.soundcloud.com/users/814464088?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 269
Get: https://api-v2.soundcloud.com/users/4127855?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 270
Get: https://api-v2.soundcloud.com/users/141030093?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 271
Get: https://api-v2.soundcloud.com/users/330273311?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 272
Get: https://api-v2.soundcloud.com/users/141025569?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
S

...

Status: Done
Total user records: 9750
Get: https://api-v2.soundcloud.com/users/236125953?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9751
Get: https://api-v2.soundcloud.com/users/85834960?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9752
Get: https://api-v2.soundcloud.com/users/876887995?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9753
Get: https://api-v2.soundcloud.com/users/176930357?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9754
Get: https://api-v2.soundcloud.com/users/158685558?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9755
Get: https://api-v2.soundcloud.com/users/173190524?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9756
Get: https://api-v2.soundcloud.com/users/101563043?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm

Status: Done
Total user records: 9809
Get: https://api-v2.soundcloud.com/users/204076698?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9810
Get: https://api-v2.soundcloud.com/users/12777624?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9811
Get: https://api-v2.soundcloud.com/users/424452900?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9812
Get: https://api-v2.soundcloud.com/users/86082214?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9813
Get: https://api-v2.soundcloud.com/users/413071827?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9814
Get: https://api-v2.soundcloud.com/users/118792460?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9815
Get: https://api-v2.soundcloud.com/users/16434139?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4q

Total user records: 9867
Get: https://api-v2.soundcloud.com/users/281135082?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9868
Get: https://api-v2.soundcloud.com/users/797481889?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9869
Get: https://api-v2.soundcloud.com/users/85239683?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9870
Get: https://api-v2.soundcloud.com/users/194494904?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9871
Get: https://api-v2.soundcloud.com/users/8994735?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9872
Get: https://api-v2.soundcloud.com/users/327017?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9873
Get: https://api-v2.soundcloud.com/users/791537740?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doi

Total user records: 9925
Get: https://api-v2.soundcloud.com/users/1913926?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9926
Get: https://api-v2.soundcloud.com/users/56457982?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9927
Get: https://api-v2.soundcloud.com/users/2042594?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9928
Get: https://api-v2.soundcloud.com/users/68297219?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9929
Get: https://api-v2.soundcloud.com/users/356156?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9930
Get: https://api-v2.soundcloud.com/users/265459607?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9931
Get: https://api-v2.soundcloud.com/users/340383341?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing


Total user records: 9983
Get: https://api-v2.soundcloud.com/users/715052434?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9984
Get: https://api-v2.soundcloud.com/users/3467583?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9985
Get: https://api-v2.soundcloud.com/users/85182?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9986
Get: https://api-v2.soundcloud.com/users/196655211?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9987
Get: https://api-v2.soundcloud.com/users/135747087?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9988
Get: https://api-v2.soundcloud.com/users/995654242?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doing
Status: Done
Total user records: 9989
Get: https://api-v2.soundcloud.com/users/175542851?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn
Status: Doi

### Lưu kết qua thu được thành file

In [6]:
user.to_csv('user.csv')

### Đọc file kết quả và xem thử phần đầu tập dữ liệu

In [7]:
df = pd.read_csv('user.csv')
df.head()

,Unnamed: 0,id,username,city,country_code,first_name,last_name,created_at,followers_count,followings_count,playlist_count,track_count,verified,likes_count,comments_count,playlist_likes_count,last_modified,avatar_url,permalink_url
0,0,630253005,Roa,Tokyo,JP,NaN,NaN,2019-05-02T08:00:39Z,40963,17,0,134,True,23,15,0,2021-10-29T11:01:14Z,https://i1.sndcdn.com/avatars-97PyQxc0CC3EjzWE...,https://soundcloud.com/roa_music1031
1,1,217441590,Free To Use Music,NaN,NaN,NaN,NaN,2016-04-06T15:07:33Z,14000,201,5,61,True,760,283,10,2021-05-31T07:59:27Z,https://i1.sndcdn.com/avatars-Z9f2ZHjqZ2RSas7J...,https://soundcloud.com/freetousemusic
2,2,324531068,Royalty Free Music - No Copyright Music,NaN,NaN,NaN,NaN,2017-08-06T09:37:27Z,12509,212,9,64,False,727,117,0,2019-04-10T18:47:43Z,https://i1.sndcdn.com/avatars-000387803255-gfw...,https://soundcloud.com/royaltyfreemusic-nocopy...
3,3,525378972,LiQWYD,NaN,NaN,NaN,NaN,2018-10-17T17:18:35Z,122746,16,7,97,True,41,231,1,2021-10-14T12:17:12Z,https://i1.sndcdn.com/avatars-GGWLITcX3rkTeohx...,https://soundcloud.com/liqwyd
4,4,9855085,Declan DP,NaN,NaN,NaN,NaN,2011-12-16T18:35:32Z,59057,9,4,59,True,1203,121,0,2021-08-20T12:15:00Z,https://i1.sndcdn.com/avatars-nyqUD36teBzn9yET...,https://soundcloud.com/declandp
